In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
import wandb

In [ ]:
# WandB Log-in
wandb.login()

In [ ]:
# Initialize the project in WandB
projectId = "temp-project"
wandb.init(project=projectId,
    notes="This is Assignment 1 for the course DA6401", 
    tags=["assignment_1", "fashion_mnist"])

In [ ]:
# Loading fashion-MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize the dataset (scale pixel values to [0, 1])
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape the data to add the channel dimension (28, 28, 1 for grayscale images)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Class names for f-MNIST dataset
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Dictionary to store one sample image for each class
sample_images = {}

# Iterating training data to collect one image per class
for img, label in zip(X_train, y_train):
    if label not in sample_images:
        sample_images[label] = img
    # Image for every class (0-9)
    if len(sample_images) == 10:
        break

# Create a grid plot (2 rows x 5 columns) to display the images
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
axes = axes.flatten()

table = wandb.Table(columns=["Image", "Label"])

for i in range(10):
    axes[i].imshow(sample_images[i], cmap='gray')
    axes[i].set_title(f"Class {i}: {class_names[i]}")
    axes[i].axis('off')
    img = sample_images[i] * 255  # Rescale for visualization
    label = class_names[i]  # Get class name
    table.add_data(wandb.Image(img), label)  # Add to table


plt.tight_layout()

# Log the plot to wandb
wandb.log({"fashion_mnist_samples": table})

plt.show()

# Finish the wandb run
wandb.finish()


In [ ]:
def relu(x):
    # ReLU: f(x) = max(0,x) (0 to +ve infinite) 
    relu = np.maximum(0, x)
    return relu

def relu_derivative(x):
    # ReLU Derivative: f(x)= {0 if x<0; 1 if x>0}  | (0. to 1.)
    relu_derivative = (x > 0).astype(float) 
    return relu_derivative

def sigmoid(x):
    # Sigmoid: f(x) = 1 / (1 + e^(-x)) | (0 to 1)
    sigmoid = 1 / (1 + np.exp(-x))
    return sigmoid

def sigmoid_derivative(x):
    # Sigmoid Derivative: f(x) = 1 / (1 + e^(-x)) * (1 - f(x)) | (0 to +ve infinite)
    sig = sigmoid(x)
    sigmoid_derivative = sig * (1 - sig)
    return sigmoid_derivative

def tanh(x):
    # Tanh: f(x) = (e^x - e^(-x)) / (e^x + e^(-x)) | (-1 to 1)
    tanh = np.tanh(x)
    return tanh

def tanh_derivative(x):
    # Tanh Derivative: f(x) = 1 - f(x)^2 | (0 to 1) 
    tanh_derivative = 1 - np.tanh(x)**2
    return tanh_derivative

def softmax(x):
    # Softmax: f(x) = e^(x - max(x)) / sum(e^(x - max(x))) | (0 to 1)
    ex_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    soft_max = ex_x / np.sum(ex_x, axis=1, keepdims=True)
    return soft_max

In [ ]:
# FeedForward Neural Network class
class FeedForwardNN:
    # Initializing Constructor
    def __init__(self, input_size, hidden_layers, output_size, activation='relu', weight_init='random'):
        self.activation_name = activation
        if activation == 'relu':
            self.activation = relu
            self.activation_derivative = relu_derivative
        elif activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_derivative = sigmoid_derivative
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_derivative = tanh_derivative
        else:
            raise ValueError("Unsupported activation function.")
        
        # Layer sizes and Initialize parameters
        self.layer_sizes = [input_size] + hidden_layers + [output_size]
        self.weights = []
        self.biases = []
        for i in range(len(self.layer_sizes) - 1):
            if weight_init == "random":
                W = np.random.randn(self.layer_sizes[i], self.layer_sizes[i+1]) * 0.01
            elif weight_init == "xavier":
                W = np.random.randn(self.layer_sizes[i], self.layer_sizes[i+1]) * np.sqrt(1.0 / self.layer_sizes[i])
            else:
                print(f"Invalid weight_init: {weight_init}")
                raise ValueError("Unsupported weight initialization method.")

            b = np.zeros((1, self.layer_sizes[i+1]))
            self.weights.append(W)
            self.biases.append(b)
    
    def forward(self, X):
        """
        Performs a forward pass and caches intermediate activations and linear transforms.
        Returns the output and a cache dictionary.
        Pre-activation values (Z): 
        (a -> Z, h -> A)
        Linear transformation at each layer : ai(x)=Wi hi-1(x)+bi

        Activations (A): 
        After applying the activation function : hi(x) = g(ai(x)) 
        """
        cache = {"A": [], "Z": []}
        A = X
        cache["A"].append(A)  # A0 = input
        
        # Hidden layers
        for i in range(len(self.weights) - 1):
            """
            Z[i]=A[i-1]W[i]+b[i]
            Z[i] is the linear transformation (input to activation) at layer i,
            A[i-1] is the output of the previous layer,
            W and be are weights and biases for layer i.
            """
            Z = np.dot(A, self.weights[i]) + self.biases[i]
            cache["Z"].append(Z)
            A = self.activation(Z)
            cache["A"].append(A)
        
        # Output layer with softmax activation function
        Z = np.dot(A, self.weights[-1]) + self.biases[-1]
        cache["Z"].append(Z)
        A = softmax(Z)
        cache["A"].append(A)
        return A, cache

    def cross_entropy_loss(self, Y_pred, Y_true):
        """
        Computes cross-entropy loss.
        
        m is the number of examples,
        Y_true is the one-hot encoded vector of true labels for the i-th example,
        Y_pred is the predicted probability distribution of the i-th example.
        """
        m = Y_true.shape[0]
        loss = -np.sum(Y_true * np.log(Y_pred + 1e-8)) / m
        return loss
    
    def compute_accuracy(self, Y_pred, Y_true):
        # Computes the accuracy of the model.

        predictions = np.argmax(Y_pred, axis=1)
        labels = np.argmax(Y_true, axis=1)
        return np.mean(predictions == labels)


    def backward(self, X, Y_true, cache):
        """
        Performs backpropagation to compute gradients for weights and biases.
        Returns lists of gradients for weights and biases.
        """
        m = X.shape[0]
        L = len(self.weights)  # Total number of layers
        grads_W = [None] * L
        grads_b = [None] * L

        # Output layer gradient
        A_final = cache["A"][-1]
        dZ = A_final - Y_true  # derivative for softmax + cross entropy (output layer)
        grads_W[L - 1] = np.dot(cache["A"][-2].T, dZ) / m # dW = A_prev * dZ/m (gradient of weights)
        grads_b[L - 1] = np.sum(dZ, axis=0, keepdims=True) / m # db = dZ/m (gradient of biases)

        # Backpropagate through hidden layers
        for i in range(L-2, -1, -1):
            dA = np.dot(dZ, self.weights[i + 1].T)
            Z = cache["Z"][i]
            dZ = dA * self.activation_derivative(Z)
            grads_W[i] = np.dot(cache["A"][i].T, dZ) / m
            grads_b[i] = np.sum(dZ, axis=0, keepdims=True) / m

        return grads_W, grads_b

    def update_parameters(self, grads_W, grads_b, optimizer):
        """
        Combines weights and biases into a single list, updates them using the provided optimizer,
        and then splits them back into weights and biases.

        W[i]=W[i]-η⋅∂Loss/∂W[i]
        """
        # Combine parameters and gradients into one list each.
        params = self.weights + self.biases
        grads = grads_W + grads_b

        # Use the optimizer to update the parameters in place.
        updated_params = optimizer.update(params, grads)

        # Split updated_params back into weights and biases.
        L = len(self.weights)
        self.weights = updated_params[:L]
        self.biases = updated_params[L:]

In [ ]:
class Optimizer:
    """
    Base Optimizer class that outlines the interface for optimization algorithms.
    params (list): List of model parameters (weights and biases).
    grads (list): List of gradients with respect to the model parameters.
    """
    def update(self, params, grads):
        raise NotImplementedError

class SGD(Optimizer):
    """
    Stochastic Gradient Descent (SGD)
    Formula:
        θ = θ - η * ∇θ
    where, 
        θ represents the model parameters (weights/biases),
        η is the learning rate, and 
        ∇θ is the gradient of the loss function.
    """
    def __init__(self, learning_rate=0.01):
        self.lr = learning_rate
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]
        return params

class Momentum(Optimizer):
    """
    Momentum optimizer class that helps accelerate SGD by considering past (history) gradients.
    Formula:
        v_t = β * v_(t-1) + ∇wt
    where,
        v_t is the velocity, 
        β is the momentum term, and 
        η is the learning rate.
    """
    def __init__(self, learning_rate=0.01, momentum=0.9):
        self.lr = learning_rate
        self.momentum = momentum
        self.v = None
    def update(self, params, grads):
        if self.v is None:
            self.v = [np.zeros_like(p) for p in params]
        for i in range(len(params)):
            self.v[i] = self.momentum * self.v[i] - self.lr * grads[i]
            params[i] += self.v[i]
        return params

class NAGD(Optimizer):
    """
    Nesterov Accelerated Gradient (NAG), improved version of momentum.
    Formula:
        v_t = β * v_(t-1) + ∇(wt - β * v_(t-1))   
    where, 
        v_t is the momentum term, 
        β is the momentum coefficient, 
        η is the learning rate.
    """
    def __init__(self, learning_rate=0.01, momentum=0.9):
        self.lr = learning_rate
        self.momentum = momentum
        self.v = None
    def update(self, params, grads):
        if self.v is None:
            self.v = [np.zeros_like(p) for p in params]
        for i in range(len(params)):
            v_prev = self.v[i].copy()
            self.v[i] = self.momentum * self.v[i] - self.lr * grads[i]
            # Nesterov update: use the gradient evaluated at the lookahead position
            params[i] += -self.momentum * v_prev + (1 + self.momentum) * self.v[i]
        return params

class RMSProp(Optimizer):
    """
    RMSProp - Root Mean Square Propagation, which adjusts the learning rate based on recent gradients' magnitude.
    Formula:
        S_t = β * S_(t-1) + (1 - β) * (∇wt)^2
        wt+1 = wt - η * ∇wt / sqrt(S_t + ε)
    where, 
        S_t is the moving average of squared gradients, 
        β is the decay term,
        η is the learning rate, and     
        ε is a small number to prevent division by zero.
    """
    def __init__(self, learning_rate=0.001, beta=0.9, epsilon=1e-8):
        self.lr = learning_rate
        self.beta = beta
        self.epsilon = epsilon
        self.S = None
    def update(self, params, grads):
        if self.S is None:
            self.S = [np.zeros_like(p) for p in params]
        for i in range(len(params)):
            self.S[i] = self.beta * self.S[i] + (1 - self.beta) * (grads[i] ** 2)
            params[i] -= self.lr * grads[i] / (np.sqrt(self.S[i]) + self.epsilon)
        return params

class Adam(Optimizer):
    """
    Adam optimizer, which combines momentum and RMSProp to adapt learning rates for each parameter.

    Formula:
        m_t = β1 * m_(t-1) + (1 - β1) * ∇wt
        v_t = β2 * v_(t-1) + (1 - β2) * (∇wt))^2
        m_t_hat = m_t / (1 - β1^t)
        v_t_hat = v_t / (1 - β2^t)
        wt = wt - η * m_t_hat / (sqrt(v_t_hat) + ε)
    where,
        m_t and v_t are moment estimates, 
        β1 and β2 are exponential decay rates, and 
        t is the time step.
    """
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None
        self.v = None
        self.t = 0
    def update(self, params, grads):
        if self.m is None:
            self.m = [np.zeros_like(p) for p in params]
            self.v = [np.zeros_like(p) for p in params]
        self.t += 1
        for i in range(len(params)):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * grads[i]
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (grads[i] ** 2)
            m_hat = self.m[i] / (1 - self.beta1 ** self.t)
            v_hat = self.v[i] / (1 - self.beta2 ** self.t)
            params[i] -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
        return params

class Nadam(Optimizer):
    """
    Nadam (Nesterov-accelerated Adaptive Moment Estimation),

    Formula:
        m_t+1 = β1 * m_t + (1 - β1) * ∇wt
        v_t+1 = β2 * v_t + (1 - β2) * (∇wt)^2
        m_t_hat = m_t+1 / (1 - β1^t+1)
        v_t_hat = v_t+1 / (1 - β2^t+1)
        wt+1 = wt - (η / (sqrt(v_t_hat) + ε)* (β1 * m_t+1_hat + (1 - β1) * ∇wt/ 1 - β1_t+1)
    where, 
        β1 and β2 are exponential decay rates, and
        t is the time step.
    """    
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None
        self.v = None
        self.t = 0
    def update(self, params, grads):
        if self.m is None:
            self.m = [np.zeros_like(p) for p in params]
            self.v = [np.zeros_like(p) for p in params]
        self.t += 1
        for i in range(len(params)):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * grads[i]
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (grads[i] ** 2)
            m_hat = self.m[i] / (1 - self.beta1 ** self.t)
            v_hat = self.v[i] / (1 - self.beta2 ** self.t)
            # Nadam uses a Nesterov-like momentum term:
            nesterov_m = (self.beta1 * m_hat) + ((1 - self.beta1) * grads[i] / (1 - self.beta1 ** self.t))
            params[i] -= self.lr * nesterov_m / (np.sqrt(v_hat) + self.epsilon)
        return params

In [ ]:
def one_hot_encode(y, num_classes):
    # Initializing a matrix of zeros with shape (number of samples, number of classes)
    one_hot = np.zeros((y.shape[0], num_classes))
    one_hot[np.arange(y.shape[0]), y] = 1
    return one_hot

def get_optimizer(name):
    if name == "sgd":
        return SGD(learning_rate=wandb.config.learning_rate)
    elif name == "momentum":
        momentum = wandb.config.get('momentum', 0.9)  # Default to 0.9 if not in config
        return Momentum(learning_rate=wandb.config.learning_rate, momentum=momentum)
    elif name == "nagd":
        momentum = wandb.config.get('momentum', 0.9)  # Default to 0.9 if not in config
        return NAGD(learning_rate=wandb.config.learning_rate, momentum=momentum)
    elif name == "rmsprop":
        beta = wandb.config.get('beta', 0.9)  # Default to 0.9 if not in config
        epsilon = wandb.config.get('epsilon', 1e-8)  # Default to 1e-8 if not in config
        return RMSProp(learning_rate=wandb.config.learning_rate, beta=beta, epsilon=epsilon)
    elif name == "adam":
        beta1 = wandb.config.get('beta1', 0.9)  # Default to 0.9 if not in config
        beta2 = wandb.config.get('beta2', 0.999)  # Default to 0.999 if not in config
        epsilon = wandb.config.get('epsilon', 1e-8)  # Default to 1e-8 if not i
        return Adam(learning_rate=wandb.config.learning_rate, beta1=beta1, beta2=beta2, epsilon=epsilon)
    elif name == "nadam":
        beta1 = wandb.config.get('beta1', 0.9)  # Default to 0.9 if not in config
        beta2 = wandb.config.get('beta2', 0.999)  # Default to 0.999 if not in config
        epsilon = wandb.config.get('epsilon', 1e-8)  # Default to 1e-8 if not i
        return Nadam(learning_rate=wandb.config.learning_rate, beta1=beta1, beta2=beta2, epsilon=epsilon)
    else:
        raise ValueError("Unsupported optimizer.")

In [ ]:
def train(model, optimizer, X_train, Y_train, epochs, batch_size):
    num_samples = X_train.shape[0]
    num_batches = num_samples // batch_size
    for epoch in range(epochs):
        # Shuffle the training data to ensure randomness and avoid overfitting
        permutation = np.random.permutation(num_samples)
        X_shuffled = X_train[permutation]
        Y_shuffled = Y_train[permutation]
        epoch_loss = 0.0

        for i in range(num_batches):
            start = i * batch_size
            end = start + batch_size
            X_batch = X_shuffled[start:end]
            Y_batch = Y_shuffled[start:end]
            Y_pred, cache = model.forward(X_batch)
            loss = model.cross_entropy_loss(Y_pred, Y_batch)
            accuracy = model.compute_accuracy(Y_pred, Y_batch)
            epoch_loss += loss
            grads_W, grads_b = model.backward(X_batch, Y_batch, cache)
            model.update_parameters(grads_W, grads_b, optimizer)  # Updated model parameters (weights and biases) using the optimizer

        # Calculate the average loss for the epoch (mean loss over all batches)
        avg_loss = epoch_loss / num_batches

        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy}")
        wandb.log({"epoch": epoch+1, "loss": avg_loss, "accuracy": accuracy})

In [ ]:
if __name__ == '__main__':
    wandb.init(
        project=projectId, 
        config={
            "input_size": 784,
            "hidden_layers": [128, 64, 32],
            "output_size": 10,
            "activation": "relu",     
            "weight_init": "random",    
            "learning_rate": 0.001,
            "momentum": 0.9,
            "beta": 0.9,               
            "beta1": 0.9,              
            "beta2": 0.999,            
            "epsilon": 1e-8,
            "optimizer": "adam",   
            "batch_size": 32,
            "epochs": 5
        })
    
    config = wandb.config
    samples = 5

    # Load and preprocessing Fashion-MNIST data
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], -1) / 255.0
    X_test = X_test.reshape(X_test.shape[0], -1) / 255.0
    Y_train = one_hot_encode(y_train, config.output_size)
    Y_test = one_hot_encode(y_test, config.output_size)

    # Initializing model and optimizer
    model = FeedForwardNN(config.input_size, config.hidden_layers, config.output_size,
                          activation=config.activation, weight_init=config.weight_init)
    optimizer = get_optimizer(config.optimizer)

    # Training the model
    train(model, optimizer, X_train, Y_train, epochs=config.epochs, batch_size=config.batch_size)

    # After training, evaluating on a few samples and log results
    Y_pred, _ = model.forward(X_train[:samples])
    for i in range(Y_pred[:samples].shape[0]):
        print("Probability distribution over 10 classes of a single sample:", Y_pred[i])
    predictions = np.argmax(Y_pred, axis=1)

    Y_test_pred = model.forward(X_test)
    predictions_test = np.argmax(Y_pred, axis=1)

    print("Predictions for first 5 training samples:", predictions)
    print("Original labels for test data:", Y_test)
    
    # Loging table of predictions
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    table = wandb.Table(columns=["Sample", "Predicted Class", "Class Name"])

    for i, pred in enumerate(predictions):
        table.add_data(i, pred, class_names[pred])
    wandb.log({"Predictions": table})

    wandb.finish()

In [ ]:
# To verify the predictions for samples
true_classes = np.argmax(Y_train[:samples], axis=1)  # True class indices

fig, axes = plt.subplots(1, samples, figsize=(15, 9))  # 1 rows, 5 columns
axes = axes.flatten()  # Flatten axes array for easy indexing

for i in range(samples):
    # Plot the image
    ax = axes[i]
    ax.imshow(X_train[i].reshape(28, 28), cmap="gray")  # Reshaping the flattened image to 28x28

    # Title showing prediction result
    pred_class = predictions[i]
    true_class = true_classes[i]
    pred_class_name = class_names[pred_class]
    true_class_name = class_names[true_class]
    
    # Check if the prediction is correct or not
    correct = 'Correct' if pred_class == true_class else 'Incorrect'

    ax.set_title(f"Pred: {pred_class_name}\nTrue: {true_class_name}\n{correct}")
    ax.axis('off')  # Hide axis

plt.tight_layout()
plt.show()

table = wandb.Table(columns=["Sample", "Predicted Class", "True Class", "Correct?"])
for i in range(samples):
    pred_class = predictions[i]
    true_class = true_classes[i]
    pred_class_name = class_names[pred_class]
    true_class_name = class_names[true_class]
    correct = 'Correct' if pred_class == true_class else 'Incorrect'

In [ ]:
def train_sweep():
    # Initialize wandb for the current run. The hyperparameters come from wandb.config.
    wandb.init()
    config = wandb.config
    run_name = f"hl_{config.hidden_layers}_bs_{config.batch_size}_ac_{config.activation.lower()}"
    print(run_name)
    wandb.run.name = run_name

    input_size = config.input_size
    output_size = config.output_size
    hidden_layers = config.hidden_layers  # Number of hidden layers
    hidden_size = config.hidden_size  # Size of each hidden layer
    activation_function = config.activation  # Activation function
    weight_init = config.weight_init  # Weight initialization method
    optimizer_name = config.optimizer  # Optimizer choice
    batch_size = config.batch_size  # Batch size
    num_epochs = config.epochs  # Number of epochs
    learning_rate = config.learning_rate  # Learning rate
    weight_decay = config.weight_decay  # Weight decay (L2 regularization)

    
    # Load Fashion-MNIST data (using the standard train/test split)
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

    # Preprocess the images: flatten and normalize to [0, 1]
    X_train = X_train.reshape(X_train.shape[0], -1) / 255.0
    X_test = X_test.reshape(X_test.shape[0], -1) / 255.0

    # Create a validation split: 90% training, 10% validation
    num_train = int(0.9 * X_train.shape[0])
    permutation = np.random.permutation(X_train.shape[0])
    X_train = X_train[permutation]
    y_train = y_train[permutation]
    X_train_split = X_train[:num_train]
    X_val_split = X_train[num_train:]
    y_train_split = y_train[:num_train]
    y_val_split = y_train[num_train:]

    # Use the local variable "output_size" instead of config.output_size
    num_classes = output_size  # typically 10
    Y_train_split = one_hot_encode(y_train_split, num_classes)
    Y_val_split = one_hot_encode(y_val_split, num_classes)
    hidden_layers = [hidden_size] * hidden_layers  # This will create a list of hidden layers

    # Instantiate the model (FeedForwardNN is assumed to be defined with backprop as in Question 3)
    model = FeedForwardNN(
        input_size=input_size,
        output_size=output_size,
        hidden_layers=hidden_layers,
        activation=activation_function.lower(),  # convert to lowercase if needed
        weight_init=weight_init.lower()
    )
    optimizer = get_optimizer(optimizer_name)
    num_samples = X_train_split.shape[0]
    num_batches = num_samples // batch_size

    best_val_loss = float('inf')
    best_epoch = -1
    training_loss = None
    # Training loop
    for epoch in range(num_epochs):
        # Shuffle training data at each epoch
        permutation = np.random.permutation(num_samples)
        X_train_epoch = X_train_split[permutation]
        Y_train_epoch = Y_train_split[permutation]
        epoch_loss = 0.0

        for i in range(num_batches):
            start = i * batch_size
            end = start + batch_size
            X_batch = X_train_epoch[start:end]
            Y_batch = Y_train_epoch[start:end]

            # Forward pass
            Y_pred, cache = model.forward(X_batch)
            loss = model.cross_entropy_loss(Y_pred, Y_batch)
            epoch_loss += loss

            # Backward pass
            grads_W, grads_b = model.backward(X_batch, Y_batch, cache)

            # Update parameters using the chosen optimizer
            model.update_parameters(grads_W, grads_b, optimizer)

        avg_loss = epoch_loss / num_batches   # training_loss = avg_loss


        # Evaluate on the validation set
        Y_val_pred, _ = model.forward(X_val_split)
        val_loss = model.cross_entropy_loss(Y_val_pred, Y_val_split)

        predictions = np.argmax(Y_val_pred, axis=1)
        correct = np.sum(predictions == y_val_split)
        val_accuracy_epoch = correct / len(y_val_split)


        # Log epoch-level metrics to wandb
        wandb.log({
            "epoch": epoch + 1,
            "loss": avg_loss,
            "val_loss": val_loss,
            "val_accuracy": val_accuracy_epoch,
        })

    print(f"Sweep run complete: Best Epoch: {best_epoch}, Best Val Loss: {best_val_loss:.4f}, Val Accuracy: {val_accuracy_epoch:.4f}")

sweep_config = {
    'method': 'bayes',  #(bayes) Using random search for efficiency in high-dimensional spaces
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'epochs': {'values': [5, 10]},
        'hidden_layers': {'values': [3, 4, 5]},  # Number of hidden layers
        'hidden_size': {'values': [32, 64, 128]},  # Size of each hidden layer
        'weight_decay': {'values': [0, 0.0005, 0.5]},  # L2 regularization factor
        'learning_rate': {'values': [0.0001, 0.001, 0.01]},
        'optimizer': {'values': ['sgd', 'momentum', 'nagd', 'rmsprop', 'adam', 'nadam']},
        'batch_size': {'values': [16, 32, 64]},
        'weight_init': {'values': ['random', 'xavier']},
        'activation': {'values': ['sigmoid', 'tanh', 'relu']},  # Use lowercase for consistency
        'input_size': {'value': 784},  # Fixed input size for Fashion-MNIST
        'output_size': {'value': 10},   # Fixed output classes
        'epsilon':{ 'values': [1e-8, 1e-7, 1e-6]},
        'beta': {'values': [0.9, 0.99, 0.999]},  # Possible values for beta
        'momentum': {'values': [0.8, 0.9, 0.95]},  # Possible values for momentum
    }
}
sweep_id = wandb.sweep(sweep_config, project=projectId)

if __name__ == '__main__':
    wandb.agent(sweep_id, function=train_sweep)